# Coleta limpa: 24 B3 (.SA) + 7 indicadores (2012-01-01 ↔ 2025-09-20)

- Persistência: Parquets individuais por série em `00_data/01_bruto`.
- Janela: do mais recente (2025-09-20) até o mais antigo, cortando em 2012-01-01.
- Fontes:
  - Ações B3: investpy (com fallback yfinance para listagens mais recentes)
  - Indicadores/ETF/commodities: yfinance


In [1]:
# 1) Instalação e imports essenciais
%pip install --quiet investpy==1.0.8 yfinance==0.2.58 lxml==4.9.3 html5lib==1.1 tqdm>=4.66.0 pyarrow>=14.0.0

import os, time
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd
from tqdm import tqdm
from zoneinfo import ZoneInfo
import investpy, yfinance as yf

TZ_SP = ZoneInfo("America/Sao_Paulo")
CUT_MAX = datetime(2025, 9, 20, tzinfo=TZ_SP).date()   # mais recente
CUT_MIN = datetime(2012, 1, 1, tzinfo=TZ_SP).date()    # mais antigo
OUT_DIR = Path(r"G:/Drives compartilhados/BOLSA_2026/a_bolsa2026_gemini/00_data/01_bruto")
OUT_DIR.mkdir(parents=True, exist_ok=True)
print("OUT_DIR =", OUT_DIR)



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


g:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\.conda\Lib\site-packages\investpy\bonds.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


OUT_DIR = G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\00_data\01_bruto


In [2]:
# 2) Lista dos 24 B3 (.SA) e 7 indicadores
TICKERS_B3 = [
    "ABEV3.SA", "B3SA3.SA", "BBAS3.SA", "CSNA3.SA", "CPLE6.SA", "ELET3.SA", "GGBR4.SA",
    "HAPV3.SA", "ITUB4.SA", "LREN3.SA", "PETR4.SA", "PRIO3.SA", "PSSA3.SA", "RAIL3.SA",
    "RDOR3.SA", "SBSP3.SA", "SUZB3.SA", "TAEE11.SA", "TIMS3.SA", "UGPA3.SA", "VALE3.SA",
    "VIVT3.SA", "WEGE3.SA", "TOTS3.SA"
]
INDICATORS = {
    "^BVSP": "_bvsp",
    "EWZ": "ewz",
    "^GSPC": "_gspc",
    "^VIX": "_vix",
    "DX-Y.NYB": "dx-y.nyb",
    "^TNX": "_tnx",
    "BZ=F": "bz=f",
}
print(len(TICKERS_B3), "B3 +", len(INDICATORS), "indicadores")

24 B3 + 7 indicadores


In [5]:
# 3) Utilitários de datas/normalização e coletores (investpy + yfinance)

def sp_date(d: datetime | str):
    if isinstance(d, str):
        return datetime.fromisoformat(d).date()
    return d.date() if isinstance(d, datetime) else d

def clamp_window(min_str="2012-01-01", max_str="2025-09-20"):
    dmin = datetime.fromisoformat(min_str).date()
    dmax = datetime.fromisoformat(max_str).date()
    return dmin, dmax

CUT_MIN, CUT_MAX = clamp_window("2012-01-01", "2025-09-20")

def to_investing_symbol(b3_symbol: str) -> str:
    return b3_symbol.upper().replace('.SA','')


def _to_dtindex_sp(values) -> pd.DatetimeIndex:
    idx = pd.DatetimeIndex(pd.to_datetime(values))
    if idx.tz is None:
        idx = idx.tz_localize(TZ_SP)
    else:
        idx = idx.tz_convert(TZ_SP)
    return idx


def normalize_df(df: pd.DataFrame, ticker_label: str) -> pd.DataFrame:
    rename = {"Date":"date","Open":"open","High":"high","Low":"low","Close":"close","Adj Close":"adj_close","Volume":"volume"}
    df = df.rename(columns=rename)
    # construir datetime_sp em TZ_SP
    if 'date' in df.columns:
        dt_idx = _to_dtindex_sp(df['date'])
    elif 'Date' in df.columns:
        dt_idx = _to_dtindex_sp(df['Date'])
    else:
        dt_idx = _to_dtindex_sp(df.index)
    out = df.reset_index(drop=True)
    ts = pd.Series(dt_idx)
    out['datetime_sp'] = ts.values
    out['date'] = pd.to_datetime(ts).dt.date.astype(str)
    out['ticker'] = ticker_label
    # recorte da janela
    out = out[(out['date'] >= CUT_MIN.strftime('%Y-%m-%d')) & (out['date'] <= CUT_MAX.strftime('%Y-%m-%d'))]
    # garantir colunas essenciais
    for c in ["open","high","low","close","volume"]:
        if c not in out.columns:
            out[c] = pd.NA
    out = out[["ticker","date","open","high","low","close","volume","datetime_sp"]]
    out.drop_duplicates(subset=["ticker","date"], inplace=True)
    return out


def fetch_b3(b3_symbol: str) -> pd.DataFrame:
    """Tenta investpy; se falhar, cai para yfinance (útil para TIMS3/RDOR3)."""
    sym = to_investing_symbol(b3_symbol)
    f_str = CUT_MIN.strftime('%d/%m/%Y'); t_str = CUT_MAX.strftime('%d/%m/%Y')
    try:
        # tenta via search_quotes
        res = investpy.search_quotes(text=sym, products=['stocks'], countries=['brazil'])
        qlist = res if isinstance(res, list) else ([res] if res else [])
        if qlist:
            pick = next((q for q in qlist if getattr(q,'symbol','').upper()==sym.upper()), qlist[0])
            df = pick.retrieve_historical_data(from_date=f_str, to_date=t_str, as_json=False, order='descending')
            if df is not None and not df.empty:
                return normalize_df(df, b3_symbol.upper())
        # fallback investpy direto
        df = investpy.get_stock_historical_data(stock=sym, country='brazil', from_date=f_str, to_date=t_str, as_json=False, order='descending')
        if df is not None and not df.empty:
            return normalize_df(df, b3_symbol.upper())
    except Exception:
        pass
    # yfinance fallback
    tkr = yf.Ticker(b3_symbol)
    df = tkr.history(start=CUT_MIN.strftime('%Y-%m-%d'), end=(CUT_MAX + timedelta(days=1)).strftime('%Y-%m-%d'), interval='1d', auto_adjust=False)
    if df is None or df.empty:
        raise RuntimeError(f"Sem dados para {b3_symbol}")
    return normalize_df(df, b3_symbol.upper())


def fetch_indicator(yf_symbol: str, db_symbol: str) -> pd.DataFrame:
    tkr = yf.Ticker(yf_symbol)
    df = tkr.history(start=CUT_MIN.strftime('%Y-%m-%d'), end=(CUT_MAX + timedelta(days=1)).strftime('%Y-%m-%d'), interval='1d', auto_adjust=False)
    if df is None or df.empty:
        raise RuntimeError(f"Sem dados para {yf_symbol}")
    return normalize_df(df, db_symbol)


def save_parquet(df: pd.DataFrame, out_path: Path):
    if df is None or df.empty:
        raise ValueError("DF vazio.")
    out_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_parquet(out_path, index=False)
    return out_path

In [6]:
# 4) Coleta B3 (24 .SA) — do mais recente (20/09/2025) ao mais antigo (>= 01/01/2012)
results = []
errors = []
for tk in tqdm(TICKERS_B3, desc='B3 .SA'):
    try:
        df = fetch_b3(tk)
        out = OUT_DIR / f"{tk.replace('.','_').lower()}_1d.parquet"
        save_parquet(df, out)
        results.append((tk, len(df), str(out)))
    except Exception as e:
        errors.append((tk, str(e)))
        time.sleep(0.5)

print('B3 OK:', len(results), 'Falhas:', len(errors))
results[:5], errors[:5]

B3 .SA: 100%|██████████| 24/24 [00:15<00:00,  1.56it/s]

B3 OK: 24 Falhas: 0


([('ABEV3.SA',
   3409,
   'G:\\Drives compartilhados\\BOLSA_2026\\a_bolsa2026_gemini\\00_data\\01_bruto\\abev3_sa_1d.parquet'),
  ('B3SA3.SA',
   3409,
   'G:\\Drives compartilhados\\BOLSA_2026\\a_bolsa2026_gemini\\00_data\\01_bruto\\b3sa3_sa_1d.parquet'),
  ('BBAS3.SA',
   3409,
   'G:\\Drives compartilhados\\BOLSA_2026\\a_bolsa2026_gemini\\00_data\\01_bruto\\bbas3_sa_1d.parquet'),
  ('CSNA3.SA',
   3409,
   'G:\\Drives compartilhados\\BOLSA_2026\\a_bolsa2026_gemini\\00_data\\01_bruto\\csna3_sa_1d.parquet'),
  ('CPLE6.SA',
   3408,
   'G:\\Drives compartilhados\\BOLSA_2026\\a_bolsa2026_gemini\\00_data\\01_bruto\\cple6_sa_1d.parquet')],
 [])

In [7]:
# 5) Coleta indicadores/ETF/commodities (7) via yfinance
res_i = []
err_i = []
for yf_sym, db_sym in tqdm(list(INDICATORS.items()), desc='Indicadores'):
    try:
        df = fetch_indicator(yf_sym, db_sym)
        out = OUT_DIR / f"{db_sym.replace('.','_').lower()}_1d.parquet"
        save_parquet(df, out)
        res_i.append((yf_sym, db_sym, len(df), str(out)))
    except Exception as e:
        err_i.append((yf_sym, str(e)))
        time.sleep(0.5)

print('Indicadores OK:', len(res_i), 'Falhas:', len(err_i))
res_i[:5], err_i[:5]

Indicadores: 100%|██████████| 7/7 [00:03<00:00,  1.95it/s]

Indicadores OK: 7 Falhas: 0


([('^BVSP',
   '_bvsp',
   3400,
   'G:\\Drives compartilhados\\BOLSA_2026\\a_bolsa2026_gemini\\00_data\\01_bruto\\_bvsp_1d.parquet'),
  ('EWZ',
   'ewz',
   3449,
   'G:\\Drives compartilhados\\BOLSA_2026\\a_bolsa2026_gemini\\00_data\\01_bruto\\ewz_1d.parquet'),
  ('^GSPC',
   '_gspc',
   3449,
   'G:\\Drives compartilhados\\BOLSA_2026\\a_bolsa2026_gemini\\00_data\\01_bruto\\_gspc_1d.parquet'),
  ('^VIX',
   '_vix',
   3449,
   'G:\\Drives compartilhados\\BOLSA_2026\\a_bolsa2026_gemini\\00_data\\01_bruto\\_vix_1d.parquet'),
  ('DX-Y.NYB',
   'dx-y.nyb',
   3450,
   'G:\\Drives compartilhados\\BOLSA_2026\\a_bolsa2026_gemini\\00_data\\01_bruto\\dx-y_nyb_1d.parquet')],
 [])

In [8]:
# 6) Inspeção de estrutura/detalhes em 01_bruto e 02_adequado (3 ações + 3 indicadores)
from pathlib import Path
import pandas as pd

BASE = Path(r"G:/Drives compartilhados/BOLSA_2026/a_bolsa2026_gemini/00_data")
SRC_FOLDERS = ["01_bruto", "02_adequado"]

# escolha das séries para amostra
STOCKS = ["ABEV3.SA", "ITUB4.SA", "PETR4.SA"]
INDICS = ["_bvsp", "ewz", "_gspc"]

def fname_for_stock(sym: str) -> str:
    return f"{sym.lower().replace('.', '_')}_1d.parquet"

def fname_for_indicator(db_label: str) -> str:
    return f"{db_label.replace('.', '_').lower()}_1d.parquet"

def summarize_parquet(folder: str, filename: str) -> dict:
    p = BASE / folder / filename
    if not p.exists():
        return {
            "folder": folder, "file": filename, "exists": False,
            "rows": 0, "date_min": None, "date_max": None,
            "columns": None, "dtypes": None, "tz_datetime_sp": None,
            "path": str(p),
        }
    df = pd.read_parquet(p)
    info = {
        "folder": folder,
        "file": filename,
        "exists": True,
        "rows": int(df.shape[0]),
        "columns": list(df.columns),
        "dtypes": {c: str(df[c].dtype) for c in df.columns},
        "date_min": str(pd.to_datetime(df["date"]).min().date()) if "date" in df.columns and not df.empty else None,
        "date_max": str(pd.to_datetime(df["date"]).max().date()) if "date" in df.columns and not df.empty else None,
        "tz_datetime_sp": None,
        "path": str(p),
    }
    if "datetime_sp" in df.columns and not df.empty:
        try:
            v = df["datetime_sp"].iloc[0]
            tz = getattr(getattr(v, "tz", None), "key", None) or str(getattr(v, "tz", None))
            info["tz_datetime_sp"] = tz
        except Exception:
            info["tz_datetime_sp"] = None
    return info

def pretty_print(info: dict):
    print(f"[{info['folder']}] {info['file']} -> exists={info['exists']}")
    print(" path:", info["path"])
    if not info["exists"]:
        print()
        return
    print(" rows:", info["rows"], "date:", info["date_min"], "->", info["date_max"])
    print(" tz(datetime_sp):", info["tz_datetime_sp"])
    print(" columns:", info["columns"])
    print(" dtypes:", info["dtypes"])
    # amostra rápida
    try:
        df = pd.read_parquet(info["path"])
        display(df.head(2))
    except Exception:
        pass
    print()

targets = []
for s in STOCKS:
    targets.append(fname_for_stock(s))
for i in INDICS:
    targets.append(fname_for_indicator(i))

for folder in SRC_FOLDERS:
    print("===", folder, "===")
    for fn in targets:
        pretty_print(summarize_parquet(folder, fn))

=== 01_bruto ===
[01_bruto] abev3_sa_1d.parquet -> exists=True
 path: G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\00_data\01_bruto\abev3_sa_1d.parquet
 rows: 3414 date: 2012-01-02 -> 2025-09-26
 tz(datetime_sp): None
 columns: ['ticker', 'date', 'open', 'high', 'low', 'close', 'volume', 'datetime_sp']
 dtypes: {'ticker': 'object', 'date': 'object', 'open': 'float64', 'high': 'float64', 'low': 'float64', 'close': 'float64', 'volume': 'int64', 'datetime_sp': 'datetime64[ns]'}


,ticker,date,open,high,low,close,volume,datetime_sp
0,ABEV3.SA,2012-01-02,10.890463,10.980400,10.746562,10.872475,119582,2012-01-02 02:00:00
1,ABEV3.SA,2012-01-03,10.892461,10.946424,10.654626,10.748561,2099952,2012-01-03 02:00:00



[01_bruto] itub4_sa_1d.parquet -> exists=True
 path: G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\00_data\01_bruto\itub4_sa_1d.parquet
 rows: 3415 date: 2012-01-02 -> 2025-09-29
 tz(datetime_sp): None
 columns: ['ticker', 'date', 'open', 'high', 'low', 'close', 'volume', 'datetime_sp']
 dtypes: {'ticker': 'object', 'date': 'object', 'open': 'float64', 'high': 'float64', 'low': 'float64', 'close': 'float64', 'volume': 'int64', 'datetime_sp': 'datetime64[ns]'}


,ticker,date,open,high,low,close,volume,datetime_sp
0,ITUB4.SA,2012-01-02,14.165285,14.177703,13.867242,14.086635,8201763,2012-01-02 02:00:00
1,ITUB4.SA,2012-01-03,14.107332,14.438490,14.107332,14.438490,15453407,2012-01-03 02:00:00



[01_bruto] petr4_sa_1d.parquet -> exists=True
 path: G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\00_data\01_bruto\petr4_sa_1d.parquet
 rows: 3415 date: 2012-01-02 -> 2025-09-29
 tz(datetime_sp): None
 columns: ['ticker', 'date', 'open', 'high', 'low', 'close', 'volume', 'datetime_sp']
 dtypes: {'ticker': 'object', 'date': 'object', 'open': 'float64', 'high': 'float64', 'low': 'float64', 'close': 'float64', 'volume': 'int64', 'datetime_sp': 'datetime64[ns]'}


...

,ticker,date,open,high,low,close,volume,datetime_sp
0,_bvsp,2012-01-03,57836.0,59288.0,57836.0,59265.0,3083000,2012-01-03 04:00:00
1,_bvsp,2012-01-04,59263.0,59519.0,58558.0,59365.0,2252000,2012-01-04 04:00:00



[02_adequado] ewz_1d.parquet -> exists=True
 path: G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\00_data\02_adequado\ewz_1d.parquet
 rows: 3449 date: 2012-01-03 -> 2025-09-19
 tz(datetime_sp): None
 columns: ['ticker', 'date', 'open', 'high', 'low', 'close', 'volume', 'datetime_sp']
 dtypes: {'ticker': 'object', 'date': 'object', 'open': 'float64', 'high': 'float64', 'low': 'float64', 'close': 'float64', 'volume': 'int64', 'datetime_sp': 'datetime64[ns]'}


,ticker,date,open,high,low,close,volume,datetime_sp
0,ewz,2012-01-03,59.099998,60.110001,59.060001,59.700001,20052500,2012-01-03 07:00:00
1,ewz,2012-01-04,59.580002,60.470001,59.560001,59.919998,11113200,2012-01-04 07:00:00



[02_adequado] _gspc_1d.parquet -> exists=True
 path: G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\00_data\02_adequado\_gspc_1d.parquet
 rows: 3449 date: 2012-01-03 -> 2025-09-19
 tz(datetime_sp): None
 columns: ['ticker', 'date', 'open', 'high', 'low', 'close', 'volume', 'datetime_sp']
 dtypes: {'ticker': 'object', 'date': 'object', 'open': 'float64', 'high': 'float64', 'low': 'float64', 'close': 'float64', 'volume': 'int64', 'datetime_sp': 'datetime64[ns]'}


,ticker,date,open,high,low,close,volume,datetime_sp
0,_gspc,2012-01-03,1258.859985,1284.619995,1258.859985,1277.060059,3943710000,2012-01-03 07:00:00
1,_gspc,2012-01-04,1277.030029,1278.729980,1268.099976,1277.300049,3592580000,2012-01-04 07:00:00


## Etapa 1 — Setup de diretórios (pedir autorização antes de executar)

> Objetivo: criar a árvore base do Silver (02_curado) e utilitários de paths.
- Raiz do projeto: `G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini`
- Diretórios: `00_dados` (entrada/processados) e `02_curado` (camada Silver e metadados)
- Subpastas de `02_curado`:
  - `silver/ohlcv_1d/`
  - `manifestos/`
  - `catalogos/`
  - `mappings/`
  - `artifacts/`
  - `minio_data/`
  - `postgres/`

> Após inserir a célula de código, peça autorização para executar.

In [10]:
# Etapa 1 — Setup de diretórios
from pathlib import Path
from datetime import datetime
from textwrap import indent
import os

# Constantes de paths (usar raw strings)
ROOT = r"G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini"
DIR_DADOS = ROOT + r"\00_dados"
DIR_CURADO = ROOT + r"\02_curado"

SUBS = [
    r"silver\ohlcv_1d",
    r"manifestos",
    r"catalogos",
    r"mappings",
    r"artifacts",
    r"minio_data",
    r"postgres",
 ]

def ensure_tree():
    created = []
    base = Path(DIR_CURADO)
    base.mkdir(parents=True, exist_ok=True)
    for sub in SUBS:
        p = base / Path(sub)
        p.mkdir(parents=True, exist_ok=True)
        created.append(str(p))
    return created

def print_tree(root: str, max_depth: int = 3):
    root_p = Path(root)
    rows = []
    for p in sorted(root_p.rglob('*')):
        try:
            depth = len(p.relative_to(root_p).parts)
        except Exception:
            continue
        if depth <= max_depth:
            rows.append(str(p))
    print("Árvore parcial (até profundidade", max_depth, "):")
    for r in rows:
        print(" -", r)

print("ROOT:", ROOT)
print("DIR_DADOS:", DIR_DADOS)
print("DIR_CURADO:", DIR_CURADO)
created_paths = ensure_tree()
print("Criados/validados:")
for c in created_paths:
    print(" -", c)
print_tree(DIR_CURADO, max_depth=3)
print("\n[INFO] Etapa 1 pronta para execução. Autoriza rodar esta célula?")

ROOT: G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini
DIR_DADOS: G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\00_dados
DIR_CURADO: G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\02_curado
Criados/validados:
 - G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\02_curado\silver\ohlcv_1d
 - G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\02_curado\manifestos
 - G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\02_curado\catalogos
 - G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\02_curado\mappings
 - G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\02_curado\artifacts
 - G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\02_curado\minio_data
 - G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\02_curado\postgres
Árvore parcial (até profundidade 3 ):
 - G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\02_curado\artifacts
 - G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\02_curado\catalogos
 - G:\Drives compartil

## Etapa 2 — Inventário e validação mínima (pedir autorização antes de executar)

> Objetivo: varrer os processados, padronizar checagens básicas (schema/tz/cutoff) e listar anomalias antes do staging Silver.
- Diretório alvo principal: `00_dados\\02_processado` (ou `00_dados\\02_processed`)
- Fallback: `00_data\\02_adequado` e `00_data\\01_bruto` se a pasta principal não existir/vazia
- Validações:
  - Colunas mínimas (case-insensitive): ticker, date, open, high, low, close, volume
  - date coerente (sem tz) e <= cutoff (ontem SP); se houver datetime com tz, normalizar para SP e extrair date
  - Duplicatas por (ticker, date)
- Saídas:
  - Tabela-resumo (top 20) com arquivo, ticker_inferido, provider_inferido, linhas, date_min, date_max, colunas
  - Lista auditável de anomalias (faltas de colunas, datas futuras, duplicatas)

> Após inserir a célula de código, peça autorização para executar.

In [11]:
# Etapa 2 — Inventário e validação mínima
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd
from zoneinfo import ZoneInfo
from tqdm import tqdm

TZ_SP = ZoneInfo("America/Sao_Paulo")
CUTOFF_SP = (datetime.now(TZ_SP).date() - timedelta(days=1))
ROOT = r"G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini"
DIR_DADOS = ROOT + r"\00_dados"
DIR_PRIMARY_1 = DIR_DADOS + r"\02_processado"
DIR_PRIMARY_2 = DIR_DADOS + r"\02_processed"
FALLBACKS = [
    r"G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\00_data\02_adequado",
    r"G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\00_data\01_bruto",
 ]

def list_parquets(base_dir: str) -> list[Path]:
    p = Path(base_dir)
    if not p.exists():
        return []
    return sorted(p.rglob("*.parquet"))

def pick_sources() -> list[Path]:
    found = list_parquets(DIR_PRIMARY_1) or list_parquets(DIR_PRIMARY_2)
    if found:
        print("[INFO] Usando diretório principal de processados.")
        return found
    for fb in FALLBACKS:
        f = list_parquets(fb)
        if f:
            print(f"[WARN] Diretório principal vazio/ausente. Usando fallback: {fb}")
            return f
    print("[ERRO] Nenhum arquivo parquet encontrado em fontes conhecidas.")
    return []

def infer_ticker_from_name(name: str) -> str | None:
    nm = name.lower()
    if nm.endswith("_1d.parquet"):
        base = nm[:-len("_1d.parquet")]
        return base.replace('_sa','').replace('_','').upper()  # heurística simples
    return None

def infer_provider_from_name(name: str) -> str | None:
    nm = name.lower()
    if any(k in nm for k in ["_bvsp","_gspc","_vix","dx-y","_tnx","bz=f","ewz"]):
        return "yahoo"
    return "investing"

REQ_COLS = ["ticker","date","open","high","low","close","volume"]

def normalize_basic(df: pd.DataFrame) -> pd.DataFrame:
    # renomear padrões conhecidos
    ren = {"Date":"date","Open":"open","High":"high","Low":"low","Close":"close","Adj Close":"adj_close","Volume":"volume"}
    df = df.rename(columns=ren)
    # garantir date como string YYYY-MM-DD sem tz (extraída de datetime_sp se existir)
    if "date" not in df.columns and "datetime_sp" in df.columns:
        s = pd.to_datetime(df["datetime_sp"])
        df["date"] = s.dt.tz_convert(TZ_SP).dt.date.astype(str) if s.dt.tz is not None else s.dt.date.astype(str)
    elif "date" in df.columns:
        s = pd.to_datetime(df["date"], errors="coerce")
        if getattr(s.dtype, "tz", None) is not None:
            s = s.dt.tz_convert(TZ_SP)
        df["date"] = s.dt.date.astype(str)
    # colunas mínimas
    for c in REQ_COLS:
        if c not in df.columns:
            df[c] = pd.NA
    # tipos numéricos
    for c in ["open","high","low","close"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df["volume"] = pd.to_numeric(df["volume"], errors="coerce")
    # cutoff
    df = df[df["date"] <= CUTOFF_SP.strftime('%Y-%m-%d')]
    return df

def scan_and_validate(paths: list[Path]) -> tuple[pd.DataFrame, list[str]]:
    rows = []
    anoms: list[str] = []
    for p in tqdm(paths, desc="Inventário"):
        try:
            df = pd.read_parquet(p)
            dfn = normalize_basic(df.copy())
            cols = list(dfn.columns)
            miss = [c for c in REQ_COLS if c not in dfn.columns]
            # duplicatas
            dups = pd.DataFrame()
            if all(c in dfn.columns for c in ["ticker","date"]):
                dups = dfn.duplicated(subset=["ticker","date"], keep=False)
            dup_count = int(dups.sum()) if hasattr(dups, 'sum') else 0
            # datas futuras (após cutoff) já removidas; relatar se havia
            fut_count = int((pd.to_datetime(df.get("date", []), errors='coerce').dt.date > CUTOFF_SP).sum()) if "date" in df.columns else 0
            rows.append({
                "arquivo": p.name,
                "path": str(p),
                "ticker_inferido": infer_ticker_from_name(p.name),
                "provider_inferido": infer_provider_from_name(p.name),
                "linhas": int(dfn.shape[0]),
                "date_min": str(pd.to_datetime(dfn["date"]).min().date()) if not dfn.empty else None,
                "date_max": str(pd.to_datetime(dfn["date"]).max().date()) if not dfn.empty else None,
                "colunas": cols,
                "miss_cols": miss,
                "dup_count": dup_count,
                "had_future_dates": fut_count,
            })
            if miss:
                anoms.append(f"[MISS] {p.name} faltando {miss}")
            if dup_count > 0:
                anoms.append(f"[DUP] {p.name} duplicatas={dup_count}")
            if fut_count > 0:
                anoms.append(f"[FUTURE] {p.name} datas_futuras={fut_count}")
        except Exception as e:
            anoms.append(f"[ERRO] {p.name}: {e}")
    inv = pd.DataFrame(rows)
    return inv, anoms

paths = pick_sources()
if not paths:
    print("Nenhum arquivo para inventariar.")
else:
    inv, anoms = scan_and_validate(paths)
    print("Resumo:")
    print(" - Arquivos:", len(inv))
    print(" - Linhas totais:", int(inv["linhas"].sum()) if not inv.empty else 0)
    print(" - Data global:", inv["date_min"].min(), "->", inv["date_max"].max())
    display(inv.head(20))
    if anoms:
        print("Anomalias:")
        for a in anoms[:200]:
            print(" ", a)
    else:
        print("Sem anomalias encontradas.")
print("\n[INFO] Etapa 2 pronta. Posso executar agora?")

[WARN] Diretório principal vazio/ausente. Usando fallback: G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\00_data\02_adequado


Inventário: 100%|██████████| 31/31 [00:00<00:00, 66.64it/s]

Resumo:
 - Arquivos: 31
 - Linhas totais: 101298
 - Data global: 2012-01-02 -> 2025-09-19


,arquivo,path,ticker_inferido,provider_inferido,linhas,date_min,date_max,colunas,miss_cols,dup_count,had_future_dates
0,_bvsp_1d.parquet,G:\Drives compartilhados\BOLSA_2026\a_bolsa202...,BVSP,yahoo,3400,2012-01-03,2025-09-19,"[ticker, date, open, high, low, close, volume,...",[],0,0
1,_gspc_1d.parquet,G:\Drives compartilhados\BOLSA_2026\a_bolsa202...,GSPC,yahoo,3449,2012-01-03,2025-09-19,"[ticker, date, open, high, low, close, volume,...",[],0,0
2,_tnx_1d.parquet,G:\Drives compartilhados\BOLSA_2026\a_bolsa202...,TNX,yahoo,3448,2012-01-03,2025-09-19,"[ticker, date, open, high, low, close, volume,...",[],0,0
3,_vix_1d.parquet,G:\Drives compartilhados\BOLSA_2026\a_bolsa202...,VIX,yahoo,3449,2012-01-03,2025-09-19,"[ticker, date, open, high, low, close, volume,...",[],0,0
4,abev3_sa_1d.parquet,G:\Drives compartilhados\BOLSA_2026\a_bolsa202...,ABEV3,investing,3409,2012-01-02,2025-09-19,"[ticker, date, open, high, low, close, volume,...",[],0,0
5,b3sa3_sa_1d.parquet,G:\Drives compartilhados\BOLSA_2026\a_bolsa202...,B3SA3,investing,3409,2012-01-02,2025-09-19,"[ticker, date, open, high, low, close, volume,...",[],0,0
6,bbas3_sa_1d.parquet,G:\Drives compartilhados\BOLSA_2026\a_bolsa202...,BBAS3,investing,3409,2012-01-02,2025-09-19,"[ticker, date, open, high, low, close, volume,...",[],0,0
7,bz=f_1d.parquet,G:\Drives compartilhados\BOLSA_2026\a_bolsa202...,BZ=F,yahoo,3432,2012-01-03,2025-09-19,"[ticker, date, open, high, low, close, volume,...",[],0,0
8,cple6_sa_1d.parquet,G:\Drives compartilhados\BOLSA_2026\a_bolsa202...,CPLE6,investing,3408,2012-01-02,2025-09-19,"[ticker, date, open, high, low, close, volume,...",[],0,0
9,csna3_sa_1d.parquet,G:\Drives compartilhados\BOLSA_2026\a_bolsa202...,CSNA3,investing,3409,2012-01-02,2025-09-19,"[ticker, date, open, high, low, close, volume,...",[],0,0


Sem anomalias encontradas.

[INFO] Etapa 2 pronta. Posso executar agora?


## Etapa 3 — Dimensões e mapeamentos (pedir autorização antes de executar)

> Objetivo: derivar catálogos a partir do inventário (Etapa 2).
- dim_asset: uma linha por série com atributos: ticker, provider, asset_class (B3_EQUITY, US_INDEX, ETF, DOLLAR, OIL, RATE, VOL), exchange (B3, US), source_file
- mappings/YAHOO_TO_INVESTING.json: mapeamento de símbolos equivalentes (e.g., "ABEV3.SA" ↔ "ABEV3"), usado em reconciliação.
- Saídas:
  - `02_curado/catalogos/dim_asset.csv`
  - `02_curado/mappings/YAHOO_TO_INVESTING.json`

> Após inserir a célula de código, peça autorização para executar.

In [12]:
# Etapa 3 — Geração de dim_asset e mappings
from pathlib import Path
import pandas as pd
import json

ROOT = r"G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini"
DIR_CURADO = ROOT + r"\02_curado"
CAT_DIR = Path(DIR_CURADO) / "catalogos"
MAP_DIR = Path(DIR_CURADO) / "mappings"
CAT_DIR.mkdir(parents=True, exist_ok=True)
MAP_DIR.mkdir(parents=True, exist_ok=True)

# Pré-requisito: variável 'inv' na memória a partir da Etapa 2 (tabela-resumo).
if 'inv' not in globals() or inv is None or inv.empty:
    raise RuntimeError("Inventário (Etapa 2) não encontrado em memória. Execute a Etapa 2 antes.")

def classify(row: pd.Series) -> tuple[str, str]:
    name = (row.get('arquivo') or '').lower()
    tk = (row.get('ticker_inferido') or '').upper()
    # Heurísticas simples por padrão de nome
    if any(k in name for k in ['_bvsp','_gspc']):
        return 'US_INDEX', 'US'
    if 'ewz' in name:
        return 'ETF', 'US'
    if 'dx-y' in name:
        return 'DOLLAR', 'US'
    if 'bz=f' in name:
        return 'OIL', 'US'
    if '_tnx' in name:
        return 'RATE', 'US'
    if '_vix' in name:
        return 'VOL', 'US'
    # default: ações B3
    return 'B3_EQUITY', 'B3'

def to_investing(yahoo_ticker: str) -> str | None:
    if not yahoo_ticker:
        return None
    s = yahoo_ticker.upper()
    if s.endswith('.SA'):
        return s.replace('.SA','')
    return None

df_dim = inv[['arquivo','path','ticker_inferido','provider_inferido','date_min','date_max','linhas']].copy()
df_dim['asset_class'], df_dim['exchange'] = zip(*df_dim.apply(classify, axis=1))
df_dim.rename(columns={'ticker_inferido':'ticker','provider_inferido':'provider','arquivo':'source_file'}, inplace=True)
df_dim.sort_values(['exchange','asset_class','ticker'], inplace=True, na_position='last')

out_dim = CAT_DIR / 'dim_asset.csv'
df_dim.to_csv(out_dim, index=False)
print('dim_asset salvo em:', out_dim)
print('linhas:', len(df_dim))

# Mapeamento Yahoo -> Investing (somente B3 equities com sufixo .SA)
yahoo_b3 = df_dim[df_dim['exchange']=='B3']['ticker'].dropna().unique().tolist()
mapping = {k: to_investing(k) for k in yahoo_b3}
mapping = {k:v for k,v in mapping.items() if v}
out_map = MAP_DIR / 'YAHOO_TO_INVESTING.json'
with open(out_map, 'w', encoding='utf-8') as f:
    json.dump(mapping, f, ensure_ascii=False, indent=2)
print('Mapping salvo em:', out_map)
print('pares:', len(mapping))

print("\n[INFO] Etapa 3 pronta para execução. Autoriza rodar esta célula?")

dim_asset salvo em: G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\02_curado\catalogos\dim_asset.csv
linhas: 31
Mapping salvo em: G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\02_curado\mappings\YAHOO_TO_INVESTING.json
pares: 0

[INFO] Etapa 3 pronta para execução. Autoriza rodar esta célula?


## Etapa 4 — Calendário por exchange (pedir autorização antes de executar)

> Objetivo: gerar calendários mínimos (business days) por exchange para validação e backfill.
- Exchanges: B3 (America/Sao_Paulo), US (America/New_York)
- Campos: exchange, date
- Saída:
  - `02_curado/catalogos/dim_calendar_b3.csv`
  - `02_curado/catalogos/dim_calendar_us.csv`

> Observação: versão inicial sem feriados; após validação, podemos incorporar feriados oficiais.

> Após inserir a célula de código, peça autorização para executar.

In [13]:
# Etapa 4 — Geração de calendários mínimos (BD)
from pathlib import Path
import pandas as pd
from datetime import datetime
from zoneinfo import ZoneInfo

ROOT = r"G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini"
DIR_CURADO = ROOT + r"\02_curado"
CAT_DIR = Path(DIR_CURADO) / "catalogos"
CAT_DIR.mkdir(parents=True, exist_ok=True)

# Usar os limites globais do inventário da Etapa 2 (ou defaults)
if 'inv' in globals() and not inv.empty:
    dmin = pd.to_datetime(inv['date_min']).min()
    dmax = pd.to_datetime(inv['date_max']).max()
else:
    dmin = pd.Timestamp('2012-01-01')
    dmax = pd.Timestamp(datetime.now().date())

def business_days(start: pd.Timestamp, end: pd.Timestamp) -> pd.DatetimeIndex:
    # Úteis simples (segunda a sexta) sem feriados por enquanto
    return pd.bdate_range(start=start, end=end, freq='C')

cal_b3 = pd.DataFrame({'exchange':'B3','date': business_days(dmin, dmax).date})
cal_us = pd.DataFrame({'exchange':'US','date': business_days(dmin, dmax).date})

p_b3 = CAT_DIR / 'dim_calendar_b3.csv'
p_us = CAT_DIR / 'dim_calendar_us.csv'
cal_b3.to_csv(p_b3, index=False)
cal_us.to_csv(p_us, index=False)
print('Calendários salvos:')
print(' -', p_b3)
print(' -', p_us)

print("\n[INFO] Etapa 4 pronta para execução. Autoriza rodar esta célula?")

Calendários salvos:
 - G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\02_curado\catalogos\dim_calendar_b3.csv
 - G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\02_curado\catalogos\dim_calendar_us.csv

[INFO] Etapa 4 pronta para execução. Autoriza rodar esta célula?


## Etapa 5 — Staging + DQ (pedir autorização antes de executar)

> Objetivo: consolidar dados limpos para staging, aplicar schema fix + cutoff + dedupe, validar contra calendário (B3/US) e persistir um staging único.
- Entrada: inventário (Etapa 2), catálogos (Etapas 3 e 4)
- Regras:
  - Colunas: ticker, date, open, high, low, close, volume, datetime_sp
  - Cutoff: ontem (São Paulo)
  - Unicidade: (ticker, date, interval='1d')
  - Calendário: descartar datas fora do calendário da exchange
- Saída:
  - `02_curado/artifacts/staging_ohlcv_1d.parquet` (PyArrow, Snappy)

> Após inserir a célula de código, peça autorização para executar.

In [14]:
# Etapa 5 — Staging + DQ (gera staging_ohlcv_1d.parquet)
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from zoneinfo import ZoneInfo

ROOT = r"G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini"
DIR_CURADO = ROOT + r"\02_curado"
ART_DIR = Path(DIR_CURADO) / "artifacts"
CAT_DIR = Path(DIR_CURADO) / "catalogos"
ART_DIR.mkdir(parents=True, exist_ok=True)

TZ_SP = ZoneInfo("America/Sao_Paulo")
CUTOFF_SP = (datetime.now(TZ_SP).date() - timedelta(days=1))
REQ_COLS = ["ticker","date","open","high","low","close","volume","datetime_sp"]

# Pré-requisitos: inv (Etapa 2), dim_asset (Etapa 3), calendars (Etapa 4)
if 'inv' not in globals() or inv is None or inv.empty:
    raise RuntimeError("Inventário (Etapa 2) ausente. Execute a Etapa 2.")

dim_asset = pd.read_csv(CAT_DIR / 'dim_asset.csv')
cal_b3 = pd.read_csv(CAT_DIR / 'dim_calendar_b3.csv') if (CAT_DIR / 'dim_calendar_b3.csv').exists() else pd.DataFrame()
cal_us = pd.read_csv(CAT_DIR / 'dim_calendar_us.csv') if (CAT_DIR / 'dim_calendar_us.csv').exists() else pd.DataFrame()

# Monta set de datas válidas por exchange
val_b3 = set(pd.to_datetime(cal_b3['date']).dt.date) if not cal_b3.empty else None
val_us = set(pd.to_datetime(cal_us['date']).dt.date) if not cal_us.empty else None

def normalize_file(path: str) -> pd.DataFrame:
    df = pd.read_parquet(path)
    # renomear padrões
    ren = {"Date":"date","Open":"open","High":"high","Low":"low","Close":"close","Adj Close":"adj_close","Volume":"volume"}
    df = df.rename(columns=ren)
    # datetime_sp
    if 'datetime_sp' in df.columns:
        s = pd.to_datetime(df['datetime_sp'], errors='coerce')
        if getattr(s.dtype, 'tz', None) is None:
            s = s.dt.tz_localize(TZ_SP)
        else:
            s = s.dt.tz_convert(TZ_SP)
    else:
        # derive from 'date' or index
        s0 = pd.to_datetime(df.get('date', df.index), errors='coerce')
        s = s0.dt.tz_localize(TZ_SP) if getattr(s0.dtype, 'tz', None) is None else s0.dt.tz_convert(TZ_SP)
    df['datetime_sp'] = s
    # date como string
    df['date'] = pd.to_datetime(df.get('date', s), errors='coerce')
    if getattr(df['date'].dtype, 'tz', None) is not None:
        df['date'] = df['date'].dt.tz_convert(TZ_SP)
    df['date'] = df['date'].dt.date.astype(str)
    # completar colunas mínimas
    for c in REQ_COLS:
        if c not in df.columns:
            df[c] = pd.NA
    # numéricos
    for c in ["open","high","low","close","volume"]:
        df[c] = pd.to_numeric(df[c], errors='coerce')
    # cutoff
    df = df[df['date'] <= CUTOFF_SP.strftime('%Y-%m-%d')]
    return df[REQ_COLS]

# Carrega e empilha todos os arquivos do inventário
dfs = []
for _, row in inv.iterrows():
    p = row['path']
    try:
        d = normalize_file(p)
        # ticker do arquivo se ausente
        if d['ticker'].isna().any() or (d['ticker'] == '').any():
            tkinf = row.get('ticker_inferido')
            if tkinf:
                d['ticker'] = tkinf
        dfs.append(d)
    except Exception as e:
        print('[WARN] Falha ao normalizar', p, '-', e)

stg = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame(columns=REQ_COLS)

# Anexa exchange via dim_asset
stg['ticker'] = stg['ticker'].astype(str)
dim_sel = dim_asset[['ticker','exchange']].drop_duplicates()
stg = stg.merge(dim_sel, on='ticker', how='left')

# Filtra por calendário (se disponível)
def in_cal(row) -> bool:
    d = pd.to_datetime(row['date']).date()
    ex = row.get('exchange') or 'B3'
    if ex == 'B3' and val_b3 is not None:
        return d in val_b3
    if ex == 'US' and val_us is not None:
        return d in val_us
    return True

stg = stg[stg.apply(in_cal, axis=1)]

# Unicidade
stg.drop_duplicates(subset=['ticker','date'], inplace=True)
stg.sort_values(['ticker','date'], inplace=True)

# Persistir staging parquet
out_path = ART_DIR / 'staging_ohlcv_1d.parquet'
table = pa.Table.from_pandas(stg)
pq.write_table(table, out_path, compression='snappy')
print('Staging salvo em:', out_path)
print('linhas:', len(stg), 'tickers:', stg['ticker'].nunique())

print("\n[INFO] Etapa 5 pronta para execução. Autoriza rodar esta célula?")

Staging salvo em: G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\02_curado\artifacts\staging_ohlcv_1d.parquet
linhas: 101298 tickers: 31

[INFO] Etapa 5 pronta para execução. Autoriza rodar esta célula?


## Etapa 6 — Write Silver particionado + MinIO (pedir autorização antes de executar)

> Objetivo: materializar a camada Silver em Parquet particionado por asset_class/ticker/ano com compressão Snappy e, opcionalmente, sincronizar para MinIO.
- Entrada: `artifacts/staging_ohlcv_1d.parquet` + `dim_asset.csv`
- Layout local: `02_curado/silver/ohlcv_1d/asset_class=<...>/ticker=<...>/year=<YYYY>/part-*.parquet`
- Sincronização MinIO (opcional nesta etapa): copiar a mesma árvore sob um bucket (ex.: `silver`) em `02_curado/minio_data` como staging para upload.

> Após inserir a célula de código, peça autorização para executar.

In [ ]:
# Etapa 6 — Materialização Silver particionada e cópia para MinIO staging
from pathlib import Path
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import shutil

ROOT = r"G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini"
DIR_CURADO = ROOT + r"\02_curado"
SILVER_DIR = Path(DIR_CURADO) / "silver" / "ohlcv_1d"
MINIO_STAGE = Path(DIR_CURADO) / "minio_data" / "silver" / "ohlcv_1d"
ART_DIR = Path(DIR_CURADO) / "artifacts"
CAT_DIR = Path(DIR_CURADO) / "catalogos"
SILVER_DIR.mkdir(parents=True, exist_ok=True)
MINIO_STAGE.mkdir(parents=True, exist_ok=True)

stg_path = ART_DIR / 'staging_ohlcv_1d.parquet'
if not stg_path.exists():
    raise FileNotFoundError("Staging não encontrado. Execute a Etapa 5.")
stg = pd.read_parquet(stg_path)
dim_asset = pd.read_csv(CAT_DIR / 'dim_asset.csv')

# Harmonização de ticker para merge confiável com dim_asset
def canon(t: str) -> str:
    if t is None or (isinstance(t, float) and pd.isna(t)):
        return ''
    s = str(t).upper()
    # Remover sufixo .SA (B3)
    if s.endswith('.SA'):
        s = s[:-3]
    # Remover caracteres não alfanuméricos
    s = ''.join(ch for ch in s if ch.isalnum())
    return s

stg['ticker_canon'] = stg['ticker'].apply(canon)
dim_asset2 = dim_asset.copy()
dim_asset2['ticker_canon'] = dim_asset2['ticker'].apply(canon)
dim_sel = dim_asset2[['ticker_canon','asset_class']].drop_duplicates()
stg = stg.merge(dim_sel, on='ticker_canon', how='left')

# Se ainda faltar classe, inferir minimamente para B3 por sufixo .SA
stg['asset_class'] = stg['asset_class'].mask(stg['asset_class'].isna() & stg['ticker'].str.upper().str.endswith('.SA'), 'B3_EQUITY')

stg['year'] = pd.to_datetime(stg['date']).dt.year

# Gravação particionada por asset_class/ticker/year
n_written = 0
parts = 0
for (ac, tk, yr), dfp in stg.groupby(['asset_class','ticker','year']):
    if pd.isna(ac) or pd.isna(tk) or pd.isna(yr):
        continue
    out_dir = SILVER_DIR / f"asset_class={ac}" / f"ticker={tk}" / f"year={int(yr)}"
    out_dir.mkdir(parents=True, exist_ok=True)
    part_path = out_dir / f"part-{int(yr)}.parquet"
    table = pa.Table.from_pandas(dfp, preserve_index=False)
    pq.write_table(table, part_path, compression='snappy')
    n_written += len(dfp)
    parts += 1

print('Silver escrito em', SILVER_DIR)
print('partições:', parts, 'linhas gravadas:', n_written)

# Copiar para árvore de staging do MinIO (opcionalmente usado por cliente/minio-client)
def copy_tree(src_root: Path, dst_root: Path):
    for src in src_root.rglob('*'):
        rel = src.relative_to(src_root)
        dst = dst_root / rel
        if src.is_dir():
            dst.mkdir(parents=True, exist_ok=True)
        else:
            dst.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(src, dst)

copy_tree(SILVER_DIR, MINIO_STAGE)
print('Cópia para MinIO staging concluída em', MINIO_STAGE)

print("\n[INFO] Etapa 6 pronta para execução. Autoriza rodar esta célula?")

Silver escrito em G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\02_curado\silver\ohlcv_1d
linhas gravadas: 0
Cópia para MinIO staging concluída em G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\02_curado\minio_data\silver\ohlcv_1d

[INFO] Etapa 6 pronta para execução. Autoriza rodar esta célula?
